In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import cv2
from gait import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from angle_keypose import angle_ims

In [ ]:
# 1,1,3,128,128 => 1-sample/batch_size, 1-channels, 3 - #frames, 128 - height, 128 - width
x = torch.rand((1,1,3,120,160))
y = torch.rand((1,1,1,64,64))

In [ ]:
x.shape,y.shape

In [ ]:
class Conv3D(nn.Module):

    '''
    Performs 3D convolution
    '''

    def __init__(self):

        super(Conv3D, self).__init__()
        self.conv1 = self._convblock(1,16,3,3,False)
        self.conv2 = self._convblock(16,32,1,3,False)
        self.conv3 = self._convblock(32,16,1,3,False)
        self.conv4 = self._convblock(16,1,1,3,True)

    def _convblock(self,in_channels,out_channels,ksized,ksize,last):
        '''
        Makes a block of layers (Conv3d,ReLU,Maxpool3d,BatchNorm3d(only if !last))
        '''

        l1 = nn.Conv3d(in_channels=in_channels,out_channels=out_channels,kernel_size=(ksized,ksize,ksize))
        l2 = nn.ReLU()
        if last:
            return nn.Sequential(l1,l2)
        else:
            l3 = nn.MaxPool3d((1, 1, 1))
            l4 = nn.BatchNorm3d(out_channels)
            return nn.Sequential(l1,l2,l3,l4)
        
        
    def forward(self,x):

        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)

        return out


In [ ]:
model = Conv3D()

In [ ]:
out = model(x)
out.shape

In [ ]:
class PEIData(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

In [20]:
for i in range(50):
    numpy_horizontal = np.hstack((images[i], images_fun[i]))
    # print('Size1: ',images[i].shape,'Size1: ',images[i].shape,'Size1: ',images[i].shape)
    cv2.imshow('compare',numpy_horizontal)
    cv2.waitKey(0)
cv2.destroyAllWindows()